# Parte 1: Ciclo de vida de un modelo - Registro / Almacenamiento del Modelo
*   **Autor:** Carolina Torres Zapata
*   **Fecha:** 2025-11-24
*   **Contexto:** En escenarios donde no se dispone de un **Model Registry** centralizado (o como mecanismo de contingencia/fallback), la operación debe ser capaz de recuperar modelos directamente desde el historial de experimentos (Tracking Server).

**Objetivo de este notebook:**
Implementar un flujo de inferencia automatizado que:
1.  **Identifique dinámicamente** el mejor modelo entrenado (Champion) basándose en métricas objetivas (AUC).
2.  Cargue el modelo a memoria directamente desde los **artefactos de MLflow**.
3.  Simule un pipeline de inferencia por lotes (*Batch Inference*) sobre nuevos datos.
4.  Genere un **reporte operativo** enriquecido para la toma de decisiones de negocio.

## 1. Importar Librerías

In [0]:
import mlflow
import mlflow.sklearn
import pandas as pd
from pyspark.sql.functions import col

## 2. Búsqueda del "Mejor Modelo" (Simulación de Registry)
En lugar de copiar y pegar manualmente el `run_id` (lo cual es propenso a errores humanos), consultamos programáticamente el MLflow Tracking Server.
Este bloque busca en el experimento `churn_experiment_ops`, ordena los modelos por la métrica `AUC` descendente y selecciona el ganador ("Champion") para esta ejecución.

In [0]:
experiment_path = "/Users/carolina.torresz@udea.edu.co/churn_experiment_ops" 
experiment = mlflow.get_experiment_by_name(experiment_path)

if experiment is None:
    print("El experimento no existe. Ejecuta el notebook 02 primero.")
else:
    # Buscar corridas, ordenar por AUC descendente
    runs = mlflow.search_runs(
        experiment_ids=[experiment.experiment_id],
        order_by=["metrics.auc DESC"]
    )
    
    best_run = runs.iloc[0]
    best_run_id = best_run.run_id
    best_auc = best_run["metrics.auc"]
    
    print(f"Mejor Run ID recuperado: {best_run_id}")
    print(f"Métrica AUC: {best_auc}")
    print(f"Artifact URI: {best_run.artifact_uri}")


Mejor Run ID recuperado: 93e7886de06745f1b6dc428763c38d40
Métrica AUC: 0.8209731586969438
Artifact URI: dbfs:/databricks/mlflow-tracking/1986444317646589/93e7886de06745f1b6dc428763c38d40/artifacts


## 3. Carga del Modelo desde Artefactos
Utilizamos el protocolo `runs:/<id>/model` de MLflow. Esto garantiza que estamos cargando exactamente el binario serializado que generó las métricas en el paso anterior, asegurando la reproducibilidad del entorno productivo.

In [0]:
model_uri = f"runs:/{best_run_id}/model"

print(f"Cargando modelo desde: {model_uri} ...")
#loaded_model = mlflow.pyfunc.load_model(model_uri)
loaded_model = mlflow.sklearn.load_model(model_uri)

print("Modelo cargado exitosamente en memoria.")

Cargando modelo desde: runs:/93e7886de06745f1b6dc428763c38d40/model ...
Modelo cargado exitosamente en memoria.


## 4. Inferencia de Prueba (Batch Inference)
Para simular un entorno real de producción:
1.  Cargamos datos "nuevos" (simulados desde la capa Silver).
2.  **Saneamiento de Schema:** Separamos los identificadores de cliente (`customerID`) de las variables predictoras (`X`). El modelo solo debe recibir las columnas con las que fue entrenado, sin ruido adicional.

### 4.1. Leer tabla Silver (Simulando datos nuevos)

In [0]:
# 1. Leer tabla Silver (Simulando datos nuevos)
table_name = "dev.silver.churn_data"
df_spark = spark.read.table(table_name)

# Tomamos una muestra de 10 clientes para la demo
df_inference = df_spark.sample(fraction=0.1, seed=42).limit(10).toPandas()

# 2. Separar Metadatos (Lo que el modelo NO debe ver)
# Guardamos ID y Label real en un dataframe aparte para el reporte final
cols_meta = ["customerID", "Churn"]
df_meta = df_inference[cols_meta].copy()

# 3. Crear X para el modelo (Solo las features procesadas)
# Borramos las columnas que no son features
X_new = df_inference.drop(columns=cols_meta)

print("--- Datos de Entrada al Modelo (X_new) ---")
#print(f"Columnas: {X_new.columns.tolist()}")
display(X_new.head(2))

--- Datos de Entrada al Modelo (X_new) ---


gender_Male,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines_No_phone_service,MultipleLines,InternetService_Fiber_optic,InternetService_No,OnlineSecurity_No_internet_service,OnlineSecurity,OnlineBackup_No_internet_service,OnlineBackup,DeviceProtection_No_internet_service,DeviceProtection,TechSupport_No_internet_service,TechSupport,StreamingTV_No_internet_service,StreamingTV,StreamingMovies_No_internet_service,StreamingMovies,Contract_One_year,Contract_Two_year,PaperlessBilling,PaymentMethod_Credit_card_automatic,PaymentMethod_Electronic_check,PaymentMethod_Mailed_check,tenure,MonthlyCharges,TotalCharges
0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.8400143297628127,1.5201556452330371,1.5684111969587773
0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.5322605130033284,1.6431286232453304,2.402200981291416


## 4.2. Ejecución de Inferencia


In [0]:
# Generamos las predicciones.

print("🤖 Ejecutando predicciones...")

# Predicción de Clase (0 = No se va, 1 = Se va)
preds = loaded_model.predict(X_new)

# Predicción de Probabilidad (0.0 a 1.0)
probs = loaded_model.predict_proba(X_new)[:, 1]

print("✅ Inferencia finalizada.")

🤖 Ejecutando predicciones...
✅ Inferencia finalizada.


## 5. Generación de Reporte de Negocio
Un modelo de ML por sí solo devuelve probabilidades crudas (0.0 - 1.0). Para soportar la operación, transformamos estos números en acciones claras:

*   **Probabilidad de Fuga:** El score crudo del modelo.
*   **Alerta de Gestión:** Regla de negocio aplicada (Threshold > 0.5) para etiquetar visualmente a los clientes de "ALTO RIESGO".

In [0]:
# Cruzamos las predicciones con el `customerID` original para que el reporte sea accionable.

# 1. Unimos todo en un reporte final
reporte = df_meta.copy() # Empezamos con ID y Valor Real

# 2. Agregamos las predicciones
reporte["Probabilidad_Fuga"] = probs.round(4)
reporte["Prediccion_Modelo"] = preds

# 3. Regla de Negocio: Alerta Visual
# Si la probabilidad es > 50%, marcamos Alerta Roja
reporte["Alerta_Gestion"] = reporte["Prediccion_Modelo"].apply(
    lambda x: "🔴 ALTO RIESGO" if x == 1 else "🟢 Cliente Seguro"
)

# 4. Visualización Final
print("--- REPORTE FINAL PARA SOPORTE A OPERACIÓN ---")
cols_mostrar = ["customerID", "Probabilidad_Fuga", "Prediccion_Modelo", "Alerta_Gestion", "Churn"]
display(reporte[cols_mostrar])

--- REPORTE FINAL PARA SOPORTE A OPERACIÓN ---


customerID,Probabilidad_Fuga,Prediccion_Modelo,Alerta_Gestion,Churn
8008-ESFLK,0.3344,0,🟢 Cliente Seguro,0
1555-DJEQW,0.2983,0,🟢 Cliente Seguro,1
0519-XUZJU,0.9054,1,🔴 ALTO RIESGO,1
4750-UKWJK,0.0104,0,🟢 Cliente Seguro,0
2516-XSJKX,0.0552,0,🟢 Cliente Seguro,0
0356-ERHVT,0.4124,0,🟢 Cliente Seguro,0
3055-OYMSE,0.3514,0,🟢 Cliente Seguro,0
6096-EGVTU,0.0308,0,🟢 Cliente Seguro,0
8319-QBEHW,0.2126,0,🟢 Cliente Seguro,0
7110-BDTWG,0.2296,0,🟢 Cliente Seguro,0


### 5.1. Leer la Data de Negocio "Legible" (Capa Silver)

In [0]:
# Esta tabla tiene 'Contract', 'InternetService', etc. en texto original
print("📥 Leyendo datos maestros de negocio...")
df_negocio = spark.read.table("dev.silver.clean_data").toPandas()
display(df_negocio.head())

📥 Leyendo datos maestros de negocio...


customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
1329-VHWNP,Female,0,No,No,7,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,No,Bank transfer (automatic),25.05,189.95,0
2984-MIIZL,Male,0,No,No,4,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,Yes,Bank transfer (automatic),74.8,321.9,1
0266-GMEAO,Male,0,Yes,Yes,72,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,Yes,Credit card (automatic),114.3,8058.55,0
5590-YRFJT,Female,0,Yes,No,20,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,24.45,482.8,1
5574-NXZIU,Male,0,No,No,63,Yes,No,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),109.2,7049.75,0


## 5.2. Predicciones y Datos de Negocio
Para que el reporte sea útil a los equipos de Marketing/Retención, cruzamos las predicciones con los datos maestros del cliente (Tabla `dev.silver.clean_data`).
Esto permite ver no solo **quién** se va a ir, sino **por qué** (ej. ver su tipo de contrato o antigüedad de forma legible), facilitando la estrategia de retención.

Este DataFrame final (`df_enriquecido`) representa la tabla que se podría guardarse en la capa **Gold** o que alimentaría un dashboard de PowerBI/Tableau para el equipo de retención de clientes.

In [0]:
df_enriquecido = pd.merge(
    reporte,
    df_negocio,
    on="customerID",
    how="left"
)

# 4. Selección de Columnas para el Reporte Final
# Seleccionamos una mezcla de métricas del modelo + datos de contexto
df_enriquecido = df_enriquecido[[
    "customerID",
    "Contract",
    "MonthlyCharges",
    "InternetService",
    "tenure",
    "Churn_x",
    "Prediccion_Modelo",
    "Probabilidad_Fuga",
    "Alerta_Gestion"
]].rename(columns={'Churn_x': 'Churn_Real'})

display(df_enriquecido)

customerID,Contract,MonthlyCharges,InternetService,tenure,Churn_Real,Prediccion_Modelo,Probabilidad_Fuga,Alerta_Gestion
8008-ESFLK,One year,110.5,Fiber optic,53,0,0,0.3344,🟢 Cliente Seguro
1555-DJEQW,Two year,114.2,Fiber optic,70,1,0,0.2983,🟢 Cliente Seguro
0519-XUZJU,Month-to-month,70.75,Fiber optic,1,1,1,0.9054,🔴 ALTO RIESGO
4750-UKWJK,One year,19.6,No,37,0,0,0.0104,🟢 Cliente Seguro
2516-XSJKX,Two year,78.45,DSL,41,0,0,0.0552,🟢 Cliente Seguro
0356-ERHVT,Month-to-month,45.9,DSL,11,0,0,0.4124,🟢 Cliente Seguro
3055-OYMSE,Month-to-month,73.8,Fiber optic,53,0,0,0.3514,🟢 Cliente Seguro
6096-EGVTU,One year,24.9,No,64,0,0,0.0308,🟢 Cliente Seguro
8319-QBEHW,One year,39.95,DSL,26,0,0,0.2126,🟢 Cliente Seguro
7110-BDTWG,Two year,47.05,DSL,71,0,0,0.2296,🟢 Cliente Seguro
